In [1]:
import torch
from torch.utils.data import DataLoader
import lava.lib.dl.slayer as slayer
from nmnist import NMNIST, SNN, augment

         Only Prophesee DVS demo will not run properly.
         Please install it from https://github.com/prophesee-ai/prophesee-automotive-dataset-toolbox


In [2]:
dir = '.'
nmnist = NMNIST(dir, 1, 300, download=True, transform=augment)
nmnist_test = NMNIST(dir, 1, 300, train=False)

In [12]:
device = torch.device('cpu')

model = SNN()
state_dict = torch.load('network.pt')
model.load_state_dict(state_dict)

net = model.to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

nmnist_loader = DataLoader(
    dataset=nmnist, batch_size=32, shuffle=True
)

nmnist_test_loader = DataLoader(
    dataset=nmnist, batch_size=32, shuffle=True
)

error = slayer.loss.SpikeRate(
    true_rate=0.2, false_rate=0.03, reduction='sum'
).to(device)

stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(
    net, error, optimizer, stats,
    classifier=slayer.classifier.Rate.predict, count_log=True
)

epochs = 2

for epoch in range(epochs):
    for i, (input, label) in enumerate(nmnist_loader):
        output, count = assistant.train(input, label)
        header = [
            'TRAIN\nEvent rate : ' +
            ', '.join([f'{c.item():.4f}' for c in count.flatten()]),
            'Output : ' + str(output[0].sum(dim=1) / output[0].sum()),
            'Label : ' + str(label[0])
        ]
        stats.print(epoch, iter=i, header=header, dataloader=nmnist_loader)

    """
    for i, (input, label) in enumerate(nmnist_test_loader):
        output, count = assistant.test(input, label)
        header = [
            'TEST\nEvent rate : ' +
            ', '.join([f'{c.item():.4f}' for c in count.flatten()]),
            'Output : ' + str(output[0].sum(dim=1) / output[0].sum()),
            'Label : ' + str(label[0])
        ]
        stats.print(epoch, iter=i, header=header, dataloader=nmnist_loader)
    """
    torch.save(net.state_dict(), dir + '/network.pt')

    stats.update()


TRAIN
Event rate : 0.0113, 0.0030, 0.0670
Output : tensor([0.0558, 0.1581, 0.0372, 0.1163, 0.0791, 0.2093, 0.0558, 0.0744, 0.0837,
        0.1302], grad_fn=<DivBackward0>)
Label : tensor(8)
Epoch    0:     0/60000 (0%)        
Train loss =     0.70316                          accuracy = 0.78125 

TRAIN
Event rate : 0.0117, 0.0024, 0.0409
Output : tensor([0.0396, 0.0198, 0.0594, 0.1188, 0.0495, 0.1881, 0.0396, 0.0396, 0.2277,
        0.2178], grad_fn=<DivBackward0>)
Label : tensor(8)
Epoch    0:    32/60000 (0%)        
Train loss =     0.61691                          accuracy = 0.71875 

TRAIN
Event rate : 0.0107, 0.0022, 0.0359
Output : tensor([0.0556, 0.0347, 0.4097, 0.1597, 0.0556, 0.1319, 0.0417, 0.0208, 0.0764,
        0.0139], grad_fn=<DivBackward0>)
Label : tensor(2)
Epoch    0:    64/60000 (0%)        
Train loss =     0.53575                          accuracy = 0.76042 

TRAIN
Event rate : 0.0118, 0.0023, 0.0377
Output : tensor([0.0647, 0.0216, 0.0360, 0.1223, 0.0288, 0.0288

In [3]:
model = SNN()
state_dict = torch.load('network.pt')
model.load_state_dict(state_dict)

/Users/lusterclaws/snn/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


<All keys matched successfully>

In [4]:
nmnist_test = NMNIST(dir, 1, 300, train=False)
nmnist_test.example_of_each_class(model)